In [1]:
#
# Copyright © 2019 Sunho Kim. All rights reserved.
#

In [2]:
cd ..

/gorani/gorani/backend/dataserver


In [3]:
# parameters
model_path = './out'
words_in_page = 250
dropout = 0.2
neuron = 64

In [4]:
import glob
import re
files = glob.glob(model_path + '/*-time-model.hdf5')
files = sorted(files)
model_files = [(int(re.search('([0-9]+)-time-model.hdf5', file, re.IGNORECASE).group(1)), file) for file in files]
print(model_files)

[(1, './out/1-time-model.hdf5'), (10, './out/10-time-model.hdf5')]


In [5]:
import tensorflow as tf
from gorani.tfmodels import time_model
from gorani.tf import FEATURE_LEN

class Model:
    def __init__(self, path):
        self.model = time_model(FEATURE_LEN, neuron, dropout)
        self.model.load_weights(path)
        self.graph = tf.get_default_graph()

    def predict(self, X):
        with self.graph.as_default():
            return self.model.predict(X)

In [6]:
from gorani.datadb import DataDB
from gorani.tf import convert_words, transform_to_input
from tensorflow.keras import backend as K
import numpy as np

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]
out = []
datadb = DataDB()
books = datadb.get_all('books')
for book in books:
    words = len(book.content)
    ps = chunks(convert_words(book.content), words_in_page)
    x = transform_to_input(ps)
    for model_file in model_files:
        model = Model(model_file[1])
        time = 0
        for i in range(x.shape[0]):
            time += model.predict(x[i:i+1,:,:])[0,0]
        print('user: {} book: {} time: {}'.format(model_file[0], book.id, time))
        datadb.add_book_estimated_time(model_file[0], book.id, words, time)

user: 1 book: 5 time: 180.64450764656067
user: 10 book: 5 time: 734.7628481388092
user: 1 book: 1 time: 1272.4791932106018
user: 10 book: 1 time: 3272.3232097625732
user: 1 book: 2 time: 154.4291074872017
user: 10 book: 2 time: 340.76755833625793
user: 1 book: 4 time: 206.29908549785614
user: 10 book: 4 time: 436.2573823928833
user: 1 book: 6 time: 343.33035373687744
user: 10 book: 6 time: 824.6146748065948
user: 1 book: 3 time: 231.39233523607254
user: 10 book: 3 time: 456.58702301979065
